# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities = pd.read_csv("output_data/city_weather_data.csv") 
columns = ["City", "Cloudiness", "Country", "Date", "Humidity", "Lat", "Lng", "Max Temp", "Wind Speed"]
cities = cities[columns]
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mataura,100,NZ,1579878400,94,-46.19,168.86,64.00,1.01
1,Rikitea,0,PF,1579878400,74,-23.12,-134.97,77.22,17.16
2,Ushuaia,90,AR,1579878400,71,-54.80,-68.30,50.00,5.82
3,Cherskiy,77,RU,1579878400,96,68.75,161.30,-36.92,3.04
4,Chimbote,15,PE,1579878401,56,-9.09,-78.58,81.21,7.00
...,...,...,...,...,...,...,...,...,...
557,Yaan,93,NG,1579879131,5,7.38,8.57,98.19,11.01
558,Taihe,98,CN,1579879131,72,30.87,105.38,42.80,7.09
559,High Rock,75,BS,1579879131,78,26.62,-78.28,73.40,8.05
560,Mabaruma,31,GY,1579879131,56,8.20,-59.78,85.24,4.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = cities[['Lat', 'Lng']]
weight = cities['Humidity'].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities = cities[cities["Max Temp"].between(70,80,inclusive=True)]
cities = cities[cities["Cloudiness"]==0]
cities = cities[cities["Wind Speed"]<10]
cities = cities.dropna()
cities

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,Basmat,0,IN,1579878401,38,19.32,77.17,76.91,3.94
15,Chui,0,UY,1579878402,53,-33.70,-53.46,78.62,6.60
35,Bredasdorp,0,ZA,1579878404,64,-34.53,20.04,77.00,9.17
164,Saldanha,0,ZA,1579878599,54,-33.01,17.94,78.80,6.93
244,Vizianagaram,0,IN,1579878668,83,18.12,83.42,75.20,2.24
258,Coquimbo,0,CL,1579878730,60,-29.95,-71.34,73.40,9.17
373,Gigaquit,0,PH,1579878870,85,9.59,125.70,77.70,5.75
387,Cayhagan,0,PH,1579878872,82,9.67,122.43,79.25,6.69
398,Ormara,0,PK,1579878873,50,25.21,64.64,71.47,4.41
410,Daman,0,IN,1579878934,48,20.42,72.85,73.58,5.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities
hotel_df["Hotel Name"]=""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Basmat,0,IN,1579878401,38,19.32,77.17,76.91,3.94,
15,Chui,0,UY,1579878402,53,-33.70,-53.46,78.62,6.60,
35,Bredasdorp,0,ZA,1579878404,64,-34.53,20.04,77.00,9.17,
164,Saldanha,0,ZA,1579878599,54,-33.01,17.94,78.80,6.93,
244,Vizianagaram,0,IN,1579878668,83,18.12,83.42,75.20,2.24,
258,Coquimbo,0,CL,1579878730,60,-29.95,-71.34,73.40,9.17,
373,Gigaquit,0,PH,1579878870,85,9.59,125.70,77.70,5.75,
387,Cayhagan,0,PH,1579878872,82,9.67,122.43,79.25,6.69,
398,Ormara,0,PK,1579878873,50,25.21,64.64,71.47,4.41,
410,Daman,0,IN,1579878934,48,20.42,72.85,73.58,5.01,


In [8]:
params = {
    "types": "lodging",
    "radius":5000,
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()

#     print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [9]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Basmat,0,IN,1579878401,38,19.32,77.17,76.91,3.94,Atish Mundada's Home
15,Chui,0,UY,1579878402,53,-33.70,-53.46,78.62,6.60,Nuevo Hotel Plaza
35,Bredasdorp,0,ZA,1579878404,64,-34.53,20.04,77.00,9.17,Bredasdorp Country Manor
164,Saldanha,0,ZA,1579878599,54,-33.01,17.94,78.80,6.93,Blue Bay Lodge
244,Vizianagaram,0,IN,1579878668,83,18.12,83.42,75.20,2.24,Working Womens Hostel
258,Coquimbo,0,CL,1579878730,60,-29.95,-71.34,73.40,9.17,Hotel San Juan
373,Gigaquit,0,PH,1579878870,85,9.59,125.70,77.70,5.75,Casa GLEBU
387,Cayhagan,0,PH,1579878872,82,9.67,122.43,79.25,6.69,Melrose Beach Resort
398,Ormara,0,PK,1579878873,50,25.21,64.64,71.47,4.41,
410,Daman,0,IN,1579878934,48,20.42,72.85,73.58,5.01,Silver Sands Beach Resort


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
symbols = gmaps.symbol_layer(
        locations, fill_color='red', stroke_color='red')
fig.add_layer(symbols)
hotel_lat, hotel_lon = locations


# Display Map
fig

Figure(layout=FigureLayout(height='420px'))